# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [72]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

'\nimport os \nf = open("/dev/null", "w")\nos.dup2(f.fileno(), 2)\nf.close()\n\nimport nltk\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\nnltk.download()\n'

### Carreguem les dades

In [73]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


### Funcions essencials

In [74]:
def get_token(sequence):
    """
    Retorna una llista de tokens.
    """
    return [[(token) for token, pos, entity in sentence] for sentence in sequence]

def get_token_POS(sequence):
    """
    Retorna una llista de tokens i el seu POS.
    """
    return [[(token, pos) for token, pos, entity in sentence] for sentence in sequence]

def get_token_entity(sequence):
    """
    Retorna una llista de tokens i les seves entitats.
    """
    return [[(token, entity) for token, pos, entity in sentence] for sentence in sequence]

In [75]:
def extract_entities(tagged_words, encoding='BIO'):
    """
    Extreu les entitats d'una llista de paraules etiquetades segons l'encoding especificat.
    
    Arguments:
        tagged_words: Una llista de tuples (word, tag), on 'word' és una paraula del text i 'tag' és la seva etiqueta.
        encoding: El tipus de codificació utilitzat per les etiquetes ('BIO', 'BIOW', 'IO').
        
    Retorna:
        Una llista de tuples (index_inicial, index_final, etiqueta) que representen les entitats trobades.
    """
    entities = []  # Llista on es guardaran les entitats trobades
    current_entity = []  # Guarda les paraules de l'entitat actual
    current_type = None  # Tipus de l'entitat actual
    current_start_index = None  # Índex d'inici de l'entitat actual

    for index, (word, tag) in enumerate(tagged_words):
        # Determina el tipus de tag
        tag_type = None if tag == 'O' else tag[2:]
        # Obté el prefix del tag
        tag_prefix = tag[0] if tag != 'O' else None

        # Finalitza l'entitat actual si es troba un tag 'O' o si canvia el tipus d'entitat
        if tag == 'O' or tag_prefix not in ['B', 'I', 'W'] or (tag_prefix == 'I' and tag_type != current_type):
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
        if tag == 'O':
            continue

        # Gestiona els canvis d'entitat segons l'encoding
        if encoding == 'IO' and tag_type != current_type:
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
            current_entity = [word]
            current_start_index = index
            current_type = tag_type
        elif tag_prefix in ['B', 'W']:
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
            current_entity = [word]
            current_start_index = index
            current_type = tag_type
            if tag_prefix == 'W':
                # Si el tag és 'W-', afegeix immediatament com a entitat de paraula única
                entities.append((current_start_index, index, current_type))
                current_entity = []
                current_type = None
        elif tag_prefix == 'I' and current_type == tag_type:
            current_entity.append(word)

    # Afegeix la darrera entitat si encara hi ha una pendent
    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities

In [76]:
def evaluate_entities(true_entities, pred_entities):
    """
    Avaluació de les entitats reconegudes comparant conjunts d'entitats.

    Args:
    true_entities (list): Llista de tuples representant les entitats reals (start, end, type).
    pred_entities (list): Llista de tuples representant les entitats predites (start, end, type).

    Returns:
    dict: Un diccionari amb les mètriques 'precision', 'recall', i 'f1_score'.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_sent, pred_sent in zip(true_entities, pred_entities):
        true_set = set(true_sent)
        pred_set = set(pred_sent)

        true_positives += len(true_set & pred_set)
        false_positives += len(pred_set - true_set)
        false_negatives += len(true_set - pred_set)

    precisio = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * precisio * recall / (precisio + recall) if (precisio + recall) > 0 else 0

    # Arrodonim els valors a 3 decimals
    precisio, recall, f1_score= round(precisio, 3), round(recall, 3), round(f1_score, 3)

    return {
        'precisio': precisio,
        'recall': recall,
        'f1_score': f1_score
    }
    

### Models predictor de  postags

In [77]:
from crf import *
import re
import string

class FeatureGetterPOS:
    def __init__(self):
        # Compila un patró regex per detectar dígits
        self._pattern = re.compile(r"\d")

    def has_digit(self, token):
        """
        Comprova si el token conté algun dígit.
        """
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        """
        Comprova si el token conté algun signe de puntuació.
        """
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        """
        Prefix del token.
        """
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        """
        Sufix del token.
        """
        return token[-n:] if len(token) > n else token
    

    def get_features(self, tokens, index):
        """
        Obté les característiques del token.
        """
        token = tokens[index]
        features = ["WORD_" + token]

        if token[0].isupper():
            features.append("CAPITALIZATION")

        if self.has_digit(token):
            features.append("HAS_NUM")

        if self.has_punctuation(token):
            features.append("PUNCTUATION")
            
        features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])
        features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])
        if index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)
        if index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

IMPORTANT: La següent cel·la de codi és computacionalment costosa d'executar, és per això que a la cel·la que es troba a continuació carreguem els models ja entrenats prèviament, i així estalviar-nos el temps d'execució que comporta tornar a entrenar els models .

In [78]:
'''
from nltk.tag import CRFTagger

# Diccionari amb les dades d'entrenament per cada idioma
train_data = {
    'esp': train_esp,
    'ned': train_ned
}

for lang in ['esp', 'ned']:
    # Obtenim les dades d'entrenament amb les etiquetes POS
    train_lang_pos = get_token_POS(train_data[lang])

    # Creem i configurem el model
    feature_getter_pos = FeatureGetterPOS() 
    ct_POS = CRFTagger(feature_func=feature_getter_pos.get_features)

    # Entrenem el model
    model_filename = f'model.crf.taggerPOS_{lang}'
    ct_POS.train(train_lang_pos, model_filename)  

    print(f'Model per {lang} entrenat i guardat com {model_filename}')
'''

Model per esp entrenat i guardat com model.crf.taggerPOS_esp
Model per ned entrenat i guardat com model.crf.taggerPOS_ned


In [79]:
from nltk.tag import CRFTagger

# Carreguem i configurem un CRFTagger per cada idioma
feature_getter_pos = FeatureGetterPOS()

ct_POS_esp = CRFTagger(feature_func=feature_getter_pos.get_features)
ct_POS_esp.set_model_file('model.crf.taggerPOS_esp')

ct_POS_ned = CRFTagger(feature_func=feature_getter_pos.get_features)
ct_POS_ned.set_model_file('model.crf.taggerPOS_ned')

Ara ja tenim els models predictors dels POS tags per cadascun dels dos idiomes:
* ct_POS_esp
* ct_POS_ned

In [80]:
# Funció per avaluar el POS tagging
def evaluate_POS(true_tags, pred_tags):
    """
    Avaluació del POS tagging comparant seqüències de tags.

    Args:
    true_tags (list): Llista de llistes amb els tags reals.
    pred_tags (list): Llista de llistes amb els tags predits.

    Returns:
    dict: Un diccionari amb les mètriques 'accuracy'.
    """
    total = 0
    correct = 0

    for true_sent, pred_sent in zip(true_tags, pred_tags):
        total += len(true_sent)
        correct += sum(1 for true_tag, pred_tag in zip(true_sent, pred_sent) if true_tag == pred_tag)

    accuracy = correct / total if total > 0 else 0

    return round(accuracy, 3)
  

# Prediccions del POS tagging - Espanyol
y_pred_POS_esp = ct_POS_esp.tag_sents(get_token(test_esp))
y_real_POS_esp = get_token_POS(test_esp)
print(f"Esp: {evaluate_POS(y_real_POS_esp, y_pred_POS_esp)}")

# Prediccions del POS tagging - Neerlandés
y_pred_POS_ned = ct_POS_ned.tag_sents(get_token(test_ned))
y_real_POS_ned = get_token_POS(test_ned)
print(f"Ned: {evaluate_POS(y_real_POS_ned, y_pred_POS_ned)}")


Esp: 0.961
Ned: 0.958


El model prediu el postags amb una accuracy al voltant d'un 96%. Com que és un bon resultat, l'utilitzarem en les feature functions.

### Model predicor d'entitats

Aquesta classe és igual que la anterior, però utilitza el model anterior per predir els postags

In [81]:
import re
import string
class FeatureGetter:
    def __init__(self, ct_POS, use_digit=True, use_punctuation=True, use_prefix=True, use_suffix=True, use_capitalization=True, use_prev_word=True, use_next_word=True):
        """
        Inicialitza el FeatureGetter amb opcions per activar o desactivar certes característiques.
        """
        self.ct_POS = ct_POS
        self.use_digit = use_digit
        self.use_punctuation = use_punctuation
        self.use_prefix = use_prefix
        self.use_suffix = use_suffix
        self.use_capitalization = use_capitalization
        self.use_prev_word = use_prev_word
        self.use_next_word = use_next_word
        self._pattern = re.compile(r"\d")

    def has_digit(self, token):
        """
        Comprova si el token conté algun dígit.
        """
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        """
        Comprova si el token conté algun signe de puntuació.
        """
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        """
        Prefix del token.
        """
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        """
        Sufix del token.
        """
        return token[-n:] if len(token) > n else token
    
    def pos_tag(self, token):
        """
        Etiqueta POS del token.
        """
        return self.ct_POS.tag([token])[0][1]


    def get_features(self, tokens, index):
        """
        Obté les característiques del token.
        """
        token = tokens[index]
        features = ["WORD_" + token]

        if self.use_capitalization and token[0].isupper():
            features.append("CAPITALIZATION")

        if self.use_digit and self.has_digit(token):
            features.append("HAS_NUM")

        if self.use_punctuation and self.has_punctuation(token):
            features.append("PUNCTUATION")

        if self.use_suffix:
            features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])

        if self.use_prefix:
            features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])

        pos_tag = self.pos_tag(token)
        features.append("POS_" + pos_tag)
        if self.use_prev_word and index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)

        if self.use_next_word and index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

### Grid Search - Optimitar Feature Functions

Realitzem un Grid Search per trobar la millor combinació de feature functions tal que es maximitza el recall.

Com que les dades inicials estan en codificació BIO, aquesta cerca en quadrícula es realitza amb les dades en aquesta mateixa codificació.

In [82]:
from itertools import product
from nltk.tag import CRFTagger

def grid_search_feature_combinations(train_data, val_data, feature_combinations):
    """
    Realitza un grid search amb un conjunt definit de combinacions de funcions de característiques per trobar la millor que maximitza el recall.
    
    Args:
    train_data (list): Dades d'entrenament en format BIO.
    val_data (list): Dades de validació en format BIO.
    feature_combinations (list): Una llista de diccionaris amb combinacions de funcions de característiques.
    
    Returns:
    tuple: La millor combinació de característiques i el seu valor de recall.
    """
    best_recall = 0
    best_combination = None
    
    train_data = get_token_entity(train_data)
    
    for options in feature_combinations:
        feature_getter = FeatureGetter(**options, ct_POS=ct_POS_esp)
        ct = CRFTagger(feature_func=feature_getter.get_features)
        
        # Entrena el model CRFTagger amb les dades d'entrenament
        ct.train(train_data, 'model.crf.tagger')
        
        # Prepara les dades de test i obtenir les prediccions
        y_real = [extract_entities(sent) for sent in get_token_entity(val_data)]
        val_tokens = get_token(val_data)
        y_pred = ct.tag_sents(val_tokens)
        y_pred_entities = [extract_entities(sent) for sent in y_pred]
        
        # Avaluació de les entitats predites respecte a les reals
        metrics = evaluate_entities(y_real, y_pred_entities)
        recall = metrics['recall']
        
        # Actualitza la millor combinació si el recall actual és millor
        if recall > best_recall:
            best_recall = recall
            best_combination = options

    return best_combination, best_recall

# Defineix les combinacions de paràmetres explícitament
feature_combinations = [
    {'use_prefix': True, 'use_suffix': True, 'use_prev_word': True, 'use_next_word': True},
    {'use_prefix': True, 'use_suffix': True, 'use_prev_word': False, 'use_next_word': False},
    {'use_prefix': False, 'use_suffix': False, 'use_prev_word': True, 'use_next_word': True}
]

#best_combination, best_recall = grid_search_feature_combinations(train_esp, val_esp, feature_combinations)
#print("Best feature combination:", best_combination)
#print("Best recall:", best_recall)

La cel·la anterior té un temps d'execució molt elevat, per la qual cosa si es desitja executar, només cal descomentar les últimes tres línies.
Per tal d'estalviar tot aquest temps de còmput, els resultats obtinguts són els següents:

* Best feature combination: {'use_prefix': True, 'use_suffix': True, 'use_prev_word': True, 'use_next_word': True}
* Best recall: 0.711

Com podem veure, la combinació que dona un recall més elevat és amb totes les feature functions actives, per la qual cosa les utilitzarem en el model final.

### Codificació BIO

In [83]:
from nltk.tag import CRFTagger
import pycrfsuite
import pandas as pd

def compare_models(train_data, test_data, feature_func=None, lang='esp'):
    """
    Entrena i evalua dos models CRFTagger: un bàsic i un altre amb la funció _get_features()
    modificada, mostrant les mètriques de precisio, recall i F1 en una taula.

    Args:
    train_data (list): Dades d'entrenament en format BIO.
    test_data (list): Dades de prova en format BIO.
    feature_func (callable, optional): Funció de característiques per al segon model.
    
    Returns:
    pd.DataFrame: Taula amb les mètriques de precisio, recall i F1 per a cada model.
    """
    # Entrenament del model bàsic
    ct_basic = CRFTagger(feature_func=None)
    ct_basic.train(train_data, f'model.crf.tagger_BIO_{lang}')
    
    # Entrenament del model modificat si feature_func és proporcionada
    ct_modified = CRFTagger(feature_func=feature_func)
    ct_modified.train(train_data, f'model.crf.tagger_BIO_{lang}')
    
    # Preparació de les dades de prova i predicció per a cada model
    y_real = [extract_entities(sent) for sent in get_token_entity(test_data)]
    y_pred_basic = ct_basic.tag_sents(get_token(test_data))
    y_pred_modified = ct_modified.tag_sents(get_token(test_data))
    
    y_pred_basic_entities = [extract_entities(sent) for sent in y_pred_basic]
    y_pred_modified_entities = [extract_entities(sent) for sent in y_pred_modified]
    
    # Avaluació de les entitats per a cada model
    metrics_basic = evaluate_entities(y_real, y_pred_basic_entities)
    metrics_modified = evaluate_entities(y_real, y_pred_modified_entities)
    
    # Creem la taula de resultats
    results = {
        'Metric': ['Precisio', 'Recall', 'F1 Score'],
        'CRFTagger Basic': [metrics_basic['precisio'], metrics_basic['recall'], metrics_basic['f1_score']],
        'CRFTagger Modified': [metrics_modified['precisio'], metrics_modified['recall'], metrics_modified['f1_score']]
    }
    return pd.DataFrame(results)

In [84]:
# Ús de la funció per a Espanyol i Neerlandès
train_esp_data = get_token_entity(train_esp)
feature_getter_esp = FeatureGetter(ct_POS=ct_POS_esp)

train_ned_data = get_token_entity(train_ned)
feature_getter_ned = FeatureGetter(ct_POS=ct_POS_ned)

# Resultats per Espanyol
results_esp = compare_models(train_esp_data, test_esp, feature_getter_esp.get_features, 'esp')
print("Resultats per Espanyol:")
print(results_esp)

# Resultats per Neerlandès
results_ned = compare_models(train_ned_data, test_ned, feature_getter_ned.get_features, 'ned')
print("Resultats per Neerlandès:")
print(results_ned)

Resultats per Espanyol:
     Metric  CRFTagger Basic  CRFTagger Modified
0  Precisio            0.741               0.783
1    Recall            0.708               0.768
2  F1 Score            0.724               0.776
Resultats per Neerlandès:
     Metric  CRFTagger Basic  CRFTagger Modified
0  Precisio            0.701               0.769
1    Recall            0.621               0.715
2  F1 Score            0.659               0.741


### Codificació IO

In [85]:
def bio_to_io(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a IO.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, io_tag) on 'io_tag' és en codificació IO per cada frase.
    """
    io_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        io_tagged_sentence = []
        for word, tag in sentence:
            if tag.startswith('B-'):
                # Canvia B- per I-
                io_tagged_sentence.append((word, 'I-' + tag[2:]))
            elif tag.startswith('I-'):
                io_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                io_tagged_sentence.append((word, 'O'))
        io_tagged_sentences.append(io_tagged_sentence)
    
    return io_tagged_sentences

Entrenar el model amb la codificació IO i avaluar els resultats

In [86]:
def evaluate_encoding(encoding, train_data, test_data, feature_getter, lang):
    """
    Processa les dades, entrena un model, fa prediccions, i evalua el rendiment.
    
    Args:
    lang (str): Tipus de codificació de les dades.
    train_data (list): Dades d'entrenament en format BIO.
    test_data (list): Dades de prova en format BIO.
    feature_getter (FeatureGetter): Instància de la classe FeatureGetter.
    lang (str): Idioma de les dades.

    Returns:
    None: Imprimeix les mètriques d'avaluació.
    """
    # Convertim les dades d'entrenament de format BIO a l'encoding
    if encoding == 'IO':
        train_data = bio_to_io(get_token_entity(train_data))
    else: # 'BIOW'
        train_data = bio_to_biow(get_token_entity(train_data))

    # Creem el model i l'entrenem amb les dades d'entrenament
    ct = CRFTagger(feature_func=feature_getter.get_features)
    ct.train(train_data, f'model.crf.tagger_{encoding}_{lang}')

    # Convertim les dades de prova de format BIO a l'encoding i fem les prediccions
    if encoding == 'IO':
        y_real = bio_to_io(get_token_entity(test_data))
        y_pred = bio_to_io(ct.tag_sents(get_token(test_data)))
    else: 
        y_real = bio_to_biow(get_token_entity(test_data))
        y_pred = bio_to_biow(ct.tag_sents(get_token(test_data)))

    # Extraiem les entitats reals i predites
    y_real_entities = [extract_entities(sent, f"{encoding}") for sent in y_real]
    y_pred_entities = [extract_entities(sent, f"{encoding}") for sent in y_pred]

    # Avaluem les entitats
    metrics = evaluate_entities(y_real_entities, y_pred_entities)
    print(f'Precisio: {metrics["precisio"]:.3f}, Recall: {metrics["recall"]:.3f}, F1 Score: {metrics["f1_score"]:.3f}')

In [87]:
# Avaluem els resultats en espanyol
train_esp_data = train_esp
test_esp_data = test_esp
feature_getter_esp = FeatureGetter(ct_POS=ct_POS_esp)  

print("Resultats per Espanyol:")
evaluate_encoding("IO", train_esp_data, test_esp_data, feature_getter_esp, 'esp')

# Avaluem els resultats en neerlandès
train_ned_data = train_ned
test_ned_data = test_ned
feature_getter_ned = FeatureGetter(ct_POS=ct_POS_ned)  

print("Resultats per Neerlandès:")
evaluate_encoding("IO", train_ned_data, test_ned_data, feature_getter_ned, 'ned')


Resultats per Espanyol:
Precisio: 0.785, Recall: 0.762, F1 Score: 0.773
Resultats per Neerlandès:
Precisio: 0.746, Recall: 0.685, F1 Score: 0.714


### Codificació BIOW

In [88]:
def bio_to_biow(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a BIOW.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, biow_tag) on 'biow_tag' és en codificació BIOW per cada frase.
    """
    biow_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        biow_tagged_sentence = []
        for i, (word, tag) in enumerate(sentence):
            if tag.startswith('B-'):
                # Comprovar si l'entitat només té una paraula (W)
                if i + 1 < len(sentence) and sentence[i + 1][1].startswith('I-'):
                    biow_tagged_sentence.append((word, 'B-' + tag[2:]))
                else:
                    biow_tagged_sentence.append((word, 'W-' + tag[2:]))
            elif tag.startswith('I-'):
                biow_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                biow_tagged_sentence.append((word, 'O'))
        biow_tagged_sentences.append(biow_tagged_sentence)
    
    return biow_tagged_sentences

Entrenar el model amb la codificació BIOW i avaluar els resultats

In [89]:
feature_getter_esp = FeatureGetter(ct_POS=ct_POS_esp)  
feature_getter_ned = FeatureGetter(ct_POS=ct_POS_ned)  

# Processar i avaluar per Espanyol
print("Resultats per Espanyol:")
evaluate_encoding("BIOW", train_esp_data, test_esp_data, feature_getter_esp, 'esp')

# Processar i avaluar per Neerlandès
print("Resultats per Neerlandès:")
evaluate_encoding("BIOW", train_ned_data, test_ned_data, feature_getter_ned, 'ned')

Resultats per Espanyol:
Precisio: 0.792, Recall: 0.282, F1 Score: 0.416
Resultats per Neerlandès:
Precisio: 0.804, Recall: 0.275, F1 Score: 0.409


### Prova amb textos reals

In [90]:
import os
import nltk
from nltk.tokenize import word_tokenize

def read_and_tokenize(file_path):
    """
    Llegeix un fitxer de text i retorna una llista de tokens.
    
    Args:
    file_path (str): Camí al fitxer de text.
    
    Returns:
    list: Llista de tokens del text.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        tokens = word_tokenize(text)
        return tokens

# Processar cada fitxer
tokens_esp = read_and_tokenize('./dades_reals/esp.txt')
tokens_ned = read_and_tokenize('./dades_reals/ned.txt')

En primer lloc, veiem com es comporta el model amb el primer capítol de "El Quijote", de Miguel de Cervantes.

In [91]:
feature_getter_pos = FeatureGetterPOS()

# Creem i configurem el CRFTagger per a espanyol
ct_esp = CRFTagger(feature_func=feature_getter_pos.get_features)
ct_esp.set_model_file('model.crf.tagger_BIO_esp')

# Realitzem la predicció de les etiquetes POS
y_pred = ct_esp.tag(tokens_esp)

# Extreiem les entitats de la predicció
entities = extract_entities(y_pred)

# Imprimim els tokens amb les seves etiquetes POS
print("Tokens etiquetats:")
for token, tag in y_pred:
    print(f"{token}: {tag}, ", end='')

Tokens etiquetats:
Miguel: B-PER, de: I-PER, Cervantes: I-PER, Saavedra: I-PER, Que: I-PER, trata: O, de: O, la: O, condición: O, y: O, ejercicio: O, del: O, famoso: O, hidalgo: O, D.: B-MISC, Quijote: I-MISC, de: I-MISC, la: I-MISC, Mancha: I-MISC, En: I-MISC, un: I-MISC, lugar: I-MISC, de: I-MISC, la: I-MISC, Mancha: I-MISC, ,: O, de: O, cuyo: O, nombre: O, no: O, quiero: O, acordarme: O, ,: O, no: O, ha: O, mucho: O, tiempo: O, que: O, vivía: O, un: O, hidalgo: O, de: O, los: O, de: O, lanza: O, en: O, astillero: O, ,: O, adarga: O, antigua: O, ,: O, rocín: O, flaco: O, y: O, galgo: O, corredor: O, .: O, Una: B-ORG, olla: O, de: O, algo: O, más: O, vaca: O, que: O, carnero: O, ,: O, salpicón: O, las: O, más: O, noches: O, ,: O, duelos: O, y: O, quebrantos: O, los: O, sábados: O, ,: O, lentejas: O, los: O, viernes: O, ,: O, algún: O, palomino: O, de: O, añadidura: O, los: O, domingos: O, ,: O, consumían: O, las: O, tres: O, partes: O, de: O, su: O, hacienda: O, .: O, El: O,

Finalment, veiem com es comporta el model neerlandés amb un text en aquest idioma, que hem obtingut dels textos preparatoris pels exàmens de nivell B1 i B2 neerlandesos penjats a Internet.

In [92]:
feature_getter_pos = FeatureGetterPOS()

# Creem i configurem el CRFTagger per a neerlandés
ct_ned = CRFTagger(feature_func=feature_getter_pos.get_features)
ct_ned.set_model_file('model.crf.tagger_BIO_ned')

# Realitzem la predicció de les etiquetes POS
y_pred = ct_ned.tag(tokens_ned)

# Extreiem les entitats de la predicció
entities = extract_entities(y_pred)

# Imprimim els tokens amb les seves etiquetes POS
print("Tokens etiquetats:")
for token, tag in y_pred:
    print(f"{token}: {tag}, ", end='')

Tokens etiquetats:
Gisteren: B-MISC, ging: I-MISC, ik: I-MISC, met: I-MISC, Marcel: I-MISC, naar: I-MISC, de: I-MISC, supermarkt: I-MISC, .: I-MISC, Wij: I-MISC, hadden: I-MISC, al: I-MISC, een: I-MISC, tijd: I-MISC, samen: I-MISC, ruzie: I-MISC, .: I-MISC, Toch: I-MISC, moesten: I-MISC, er: I-MISC, boodschappen: I-MISC, worden: I-MISC, gedaan: I-MISC, ,: I-MISC, dus: I-MISC, besloten: I-MISC, wij: I-MISC, om: I-MISC, toch: I-MISC, maar: I-MISC, gewoon: I-MISC, wat: I-MISC, inkopen: I-MISC, te: I-MISC, gaan: I-MISC, doen: I-MISC, .: I-MISC, We: I-MISC, kregen: I-MISC, al: I-MISC, meer: I-MISC, ruzie: I-MISC, toen: I-MISC, we: I-MISC, besloten: I-MISC, naar: I-MISC, welke: I-MISC, supermarkt: I-MISC, te: I-MISC, gaan: I-MISC, .: I-MISC, Marcel: I-MISC, wilde: I-MISC, namelijk: I-MISC, naar: I-MISC, de: I-MISC, supermarkt: I-MISC, waar: I-MISC, wasmiddel: I-MISC, in: I-MISC, de: I-MISC, aanbieding: I-MISC, was: I-MISC, en: I-MISC, ik: I-MISC, wilde: I-MISC, naar: I-MISC, de: I-MISC, supe